In [1]:
# pip install selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from tqdm.auto import tqdm 
import time
import re
import pandas as pd


c:\Users\piano\anaconda3\envs\hyojung_2015479\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
artist_links = pd.read_csv('artists_split1.csv')
artist_links.head()

,artist_link
0,https://artfacts.net/artist/elinor-bellingham-...
1,https://artfacts.net/artist/frank-brangwyn/30490
2,https://artfacts.net/artist/stephen-conroy/1625
3,https://artfacts.net/artist/hans-feibusch/30758
4,https://artfacts.net/artist/james-flewitt-mull...


In [13]:
artist_links = artist_links.dropna()
artist_links.shape

(5000, 1)

In [14]:
artist_links["artist_link"] = artist_links["artist_link"].apply(lambda x: x + "/exhibitions")

exhibitions = artist_links.copy()

In [15]:
hrefs = exhibitions['artist_link'].dropna()
hrefs

0       https://artfacts.net/artist/elinor-bellingham-...
1       https://artfacts.net/artist/frank-brangwyn/304...
2       https://artfacts.net/artist/stephen-conroy/162...
3       https://artfacts.net/artist/hans-feibusch/3075...
4       https://artfacts.net/artist/james-flewitt-mull...
                              ...                        
4995    https://artfacts.net/artist/alberto-biasi/2543...
4996    https://artfacts.net/artist/milivoj-bijelic/25...
4997    https://artfacts.net/artist/simon-bogojevic-na...
4998    https://artfacts.net/artist/zoe-borelli/25365/...
4999    https://artfacts.net/artist/zdravko-bregovac/1...
Name: artist_link, Length: 5000, dtype: object

In [16]:
def collect_museum_collections_info():
    try:
        section_header = driver.find_element(By.XPATH, "//h3[text()='Museum Collections']")
        artist_card_list = section_header.find_element(By.XPATH, "./ancestor::div[contains(@class, 'app-js-components-ArtistCard-ArtistCard__list')]")
        section_containers = artist_card_list.find_elements(By.XPATH, ".//div[contains(@class, 'app-js-components-LabeledItemList-LabeledItemList__container')]")

        all_items = []
        pending_items = []

        for container in section_containers:
            labels_and_items = container.find_elements(By.XPATH, ".//div[contains(@class, 'app-js-components-LabeledItemList-LabeledItemList__label') or contains(@class, 'app-js-components-LabeledItemList-LabeledItemList__item')]")

            for element in labels_and_items:
                if "app-js-components-LabeledItemList-LabeledItemList__label" in element.get_attribute("class"):
                    current_country = element.text.strip()
                    
                    # Assign the current label to all pending items
                    for item in pending_items:
                        item['Country'] = current_country
                        all_items.append(item)
                    pending_items.clear()
                elif "app-js-components-LabeledItemList-LabeledItemList__item" in element.get_attribute("class"):
                    li_elements = element.find_elements(By.TAG_NAME, "li")

                    for li_element in li_elements:
                        span_elements = li_element.find_elements(By.TAG_NAME, "span")
                        link = li_element.find_element(By.TAG_NAME, "a").get_attribute('href')

                        if len(span_elements) >= 2:
                            name = span_elements[0].text.strip()
                            city = span_elements[1].text.strip()
                        elif len(span_elements) == 1:
                            name = span_elements[0].text.strip()
                            city = 'N/A'
                        else:
                            name = li_element.text.strip()
                            city = 'N/A'

                        pending_items.append({
                            'Country': None,  # Placeholder for later label assignment
                            'Name': name,
                            'City': city,
                            'url': link
                        })

        # Assign the last known label to any remaining pending items
        for item in pending_items:
            item['Country'] = current_country
            all_items.append(item)

        return all_items
    except Exception as e:
        return []

def collect_section_info(section_title):
    try:
        section = driver.find_element(By.XPATH, f"//div[h3[text()='{section_title}']]")
        items_container = section.find_elements(By.CLASS_NAME, "app-js-components-LabeledItemList-LabeledItemList__container")
        
        all_items = []
        pending_items = []
        current_year = None
        
        for container in items_container:
            labels_and_items = container.find_elements(By.XPATH, ".//div[contains(@class, 'app-js-components-LabeledItemList-LabeledItemList__label') or contains(@class, 'app-js-components-LabeledItemList-LabeledItemList__item')]")
            
            for element in labels_and_items:
                if "app-js-components-LabeledItemList-LabeledItemList__label" in element.get_attribute("class"):
                    current_year = element.text.strip()
                    
                    # Add pending items with the current year
                    for item in pending_items:
                        item['Year'] = current_year
                        all_items.append(item)
                    pending_items.clear()
                    
                elif "app-js-components-LabeledItemList-LabeledItemList__item" in element.get_attribute("class"):
                    li_elements = element.find_elements(By.TAG_NAME, "li")
                    for li_element in li_elements:
                        a_tags = li_element.find_elements(By.TAG_NAME, "a")
                        if len(a_tags) >= 4:
                            try:
                                name = a_tags[0].get_attribute('innerText')
                            except:
                                name = a_tags[0].text
                            try:
                                institution = a_tags[1].get_attribute('innerText')
                            except:
                                institution = a_tags[1].text
                            try:
                                city = a_tags[2].get_attribute('innerText')
                            except:
                                city = a_tags[2].text
                            try:
                                country = a_tags[3].get_attribute('innerText')
                            except:
                                country = a_tags[3].text
                            pending_items.append({
                                'Year': current_year,  # Will be replaced when the next label is found
                                'Name': name,
                                'Institution': institution,
                                'City': city,
                                'Country': country,
                                'url': a_tags[0].get_attribute('href')
                            })
        
        # Add any remaining pending items
        for item in pending_items:
            item['Year'] = current_year
            all_items.append(item)
        
        return all_items
    except Exception as e:
        return []

def collect_dealer_directory_info():
    try:
        section = driver.find_element(By.XPATH, "//div[h3[text()='Dealer Directory']]")
        items_container = section.find_elements(By.CLASS_NAME, "app-js-components-LabeledItemList-LabeledItemList__container")

        all_items = []
        for container in items_container:
            labels = container.find_elements(By.CLASS_NAME, "app-js-components-LabeledItemList-LabeledItemList__label")
            items = container.find_elements(By.CLASS_NAME, "app-js-components-LabeledItemList-LabeledItemList__item")
            for label, item in zip(labels, items):
                all_items.append({
                    'Country': label.text,
                    'Name': item.text,
                    'url': item.find_element(By.TAG_NAME, "a").get_attribute('href')
                })
        return all_items
    except Exception as e:
        return []

def collect_catalogs_info():
    try:
        section = driver.find_element(By.XPATH, "//div[h3[text()='Catalogs']]")
        items_container = section.find_elements(By.CLASS_NAME, "app-js-components-LabeledItemList-LabeledItemList__container")

        all_items = []
        for container in items_container:
            labels = container.find_elements(By.CLASS_NAME, "app-js-components-LabeledItemList-LabeledItemList__label")
            items = container.find_elements(By.CLASS_NAME, "app-js-components-LabeledItemList-LabeledItemList__item")
            for label, item in zip(labels, items):
                all_items.append({
                    'Year': label.text,
                    'Name': item.text,
                    'url': item.find_element(By.TAG_NAME, "a").get_attribute('href')
                })
        return all_items
    except Exception as e:
        return []
    

def collect_exhibitions_info(section_title):
    try:
        section = driver.find_element(By.XPATH, f"//div[h3[text()='{section_title}']]")
        items_container = section.find_elements(By.CLASS_NAME, "app-js-components-LabeledItemList-LabeledItemList__container")
        
        all_items = []
        pending_items = []
        current_year = None
        
        for container in items_container:
            labels_and_items = container.find_elements(By.XPATH, ".//div[contains(@class, 'app-js-components-LabeledItemList-LabeledItemList__label') or contains(@class, 'app-js-components-LabeledItemList-LabeledItemList__item')]")
            
            for element in labels_and_items:
                if "app-js-components-LabeledItemList-LabeledItemList__label" in element.get_attribute("class"):
                    current_year = element.text.strip()
                    
                    # Add pending items with the current year
                    for item in pending_items:
                        item['Year'] = current_year
                        all_items.append(item)
                    pending_items.clear()
                    
                elif "app-js-components-LabeledItemList-LabeledItemList__item" in element.get_attribute("class"):
                    li_elements = element.find_elements(By.TAG_NAME, "li")
                    for li_element in li_elements:
                        a_tags = li_element.find_elements(By.TAG_NAME, "a")
                        if len(a_tags) >= 4:
                            try:
                                name = a_tags[0].get_attribute('innerText')
                            except:
                                name = a_tags[0].text
                            try:
                                institution = a_tags[1].get_attribute('innerText')
                            except:
                                institution = a_tags[1].text
                            try:
                                city = a_tags[2].get_attribute('innerText')
                            except:
                                city = a_tags[2].text
                            try:
                                country = a_tags[3].get_attribute('innerText')
                            except:
                                country = a_tags[3].text

                            # Group Exhibition Type
                            exhibition_type = 'Group Exhibition' if 'Group Exhibition' in li_element.text else 'Solo Exhibition'

                            pending_items.append({
                                'Year': current_year,  # Will be replaced when the next label is found
                                'Name': name,
                                'Institution': institution,
                                'City': city,
                                'Country': country,
                                'Type': exhibition_type,  # Group or Solo Exhibition
                                'url': a_tags[0].get_attribute('href')
                            })
        
        # Add any remaining pending items
        for item in pending_items:
            item['Year'] = current_year
            all_items.append(item)
        
        return all_items
    except Exception as e:
        return []




In [17]:

# 로그인 세션 생성 함수
def create_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--incognito')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1920,1080')
    options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')

    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    driver.get(hrefs[0])
    time.sleep(2)  # 페이지가 로드되도록 대기

    # 로그인 옵션 클릭
    login_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[text()='Login']")))
    driver.execute_script("arguments[0].click();", login_link)

    # 로그인 세부사항 입력 및 제출
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "email"))).send_keys('wkyoung2@gmail.com')
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "password"))).send_keys('!BAwk123123')

    # login 버튼 선택 및 클릭
    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, "app-js-components-AuthenticationModal-AuthenticationModal__button"))
    )
    driver.execute_script("arguments[0].click();", login_button)

    time.sleep(5)  # 로그인 후 페이지가 로드되도록 충분한 시간 대기
    return driver

# 데이터 수집 함수
def collect_data(driver, hrefs, all_info):
    for href in tqdm(hrefs):
        
        driver.get(href)
        
        # 특정 요소가 로드될 때까지 대기 (최대 5초)
        try:
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="application-root"]/div/main/div/div[2]/div/div[2]'))
            )
        except TimeoutException:
            print(f"{href} - 요소를 찾을 수 없음, 다음 URL로 이동")
            continue

        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1.5)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        try:
            call_to_action_icons = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, "app-js-components-CallToAction-CallToAction__iconLink"))
            )
        except Exception:
            call_to_action_icons = []

        for call_to_action_icon in call_to_action_icons:
            try:
                driver.execute_script("arguments[0].click();", call_to_action_icon)
                time.sleep(0.5)
            except Exception:
                pass

        time.sleep(1.5)

        # 아티스트 이름 불러오기
        label = driver.find_element(By.XPATH, '//*[@id="application-root"]/div/main/div/div[1]/div[1]/div/h1').text

        exhibitions = collect_exhibitions_info('Exhibitions') or []
        biennials = collect_section_info('Biennials') or []
        art_fairs = collect_section_info('Art fairs') or []
        museum_collections = collect_museum_collections_info() or []
        dealer_directory = collect_dealer_directory_info() or []
        catalogs = collect_catalogs_info() or []

        data = {
            'exhibitions': exhibitions,
            'biennials': biennials,
            'art_fairs': art_fairs,
            'museum_collections': museum_collections,
            'dealer_directory': dealer_directory,
            'catalogs': catalogs,
            'label': label,
        }

        label = data['label']

        item_counts = {
            'exhibitions': sum(1 for _ in data['exhibitions']),
            'biennials': sum(1 for _ in data['biennials']),
            'art_fairs': sum(1 for _ in data['art_fairs']),
            'museum_collections': sum(1 for _ in data['museum_collections']),
            'dealer_directory': sum(1 for _ in data['dealer_directory']),
            'catalogs': sum(1 for _ in data['catalogs'])
        }

        if data['exhibitions']:
            for exhibition in data['exhibitions']:
                all_info.append({'Artist': label, **exhibition})

        if data['biennials']:
            for exhibition in data['biennials']:
                all_info.append({'Artist': label, 'Type': 'Biennials', **exhibition})

        if data['art_fairs']:
            for fair in data['art_fairs']:
                if fair:
                    all_info.append({'Artist': label, 'Type': 'Art Fairs', **fair})

        if data['museum_collections']:
            for collection in data['museum_collections']:
                if collection:
                    all_info.append({'Artist': label, 'Type': 'Museum Collections', **collection})

        if data['dealer_directory']:
            for dealer in data['dealer_directory']:
                if dealer:
                    all_info.append({'Artist': label, 'Type': 'Dealer Directory', **dealer})

        if data['catalogs']:
            for catalog in data['catalogs']:
                if catalog:
                    all_info.append({'Artist': label, 'Type': 'Catalogs', **catalog})

        time.sleep(1)
        print(f"{href} collected with items: {item_counts}")

# # 모든 수집된 데이터를 저장할 리스트
# all_info = []

# # 드라이버 생성 및 로그인
# driver = create_driver()
# print("로그인 테스트 완료")

# # 데이터 수집 실행
# collect_data(driver, hrefs, all_info)

# # 완료 후 드라이버 종료
# driver.quit()

# 전역변수로 all_info 설정
all_info = []

# 100개씩 수집하고 저장하는 함수
def collect_and_save_in_batches(driver, hrefs, batch_size=100):
    global all_info
    num_batches = (len(hrefs) // batch_size) + 1
    for i in range(num_batches):
        batch_hrefs = hrefs[i * batch_size:(i + 1) * batch_size]
        if len(batch_hrefs) == 0:
            continue
        
        collect_data(driver, batch_hrefs, all_info)
        
        df_batch = pd.DataFrame(all_info[-batch_size:])  # 최근 batch_size개의 데이터를 데이터프레임으로 변환
        globals()[f'df_{i+1}'] = df_batch
        print(f'df_{i+1} 저장 완료')

        
# 드라이버 생성 및 로그인
driver = create_driver()
print("로그인 테스트 완료")

# 데이터 수집 실행
collect_and_save_in_batches(driver, hrefs)

# 완료 후 드라이버 종료
driver.quit()

로그인 테스트 완료


  1%|          | 1/100 [00:17<28:17, 17.14s/it]

https://artfacts.net/artist/elinor-bellingham-smith/102823/exhibitions collected with items: {'exhibitions': 4, 'biennials': 0, 'art_fairs': 0, 'museum_collections': 2, 'dealer_directory': 0, 'catalogs': 0}


  2%|▏         | 2/100 [00:28<22:49, 13.98s/it]

https://artfacts.net/artist/frank-brangwyn/30490/exhibitions collected with items: {'exhibitions': 65, 'biennials': 0, 'art_fairs': 1, 'museum_collections': 29, 'dealer_directory': 0, 'catalogs': 0}


...

https://artfacts.net/artist/simon-bogojevic-narath/25428/exhibitions collected with items: {'exhibitions': 9, 'biennials': 0, 'art_fairs': 0, 'museum_collections': 1, 'dealer_directory': 0, 'catalogs': 0}


 99%|█████████▉| 99/100 [18:03<00:08,  8.60s/it]

https://artfacts.net/artist/zoe-borelli/25365/exhibitions - 요소를 찾을 수 없음, 다음 URL로 이동


100%|██████████| 100/100 [18:10<00:00, 10.90s/it]

https://artfacts.net/artist/zdravko-bregovac/170575/exhibitions collected with items: {'exhibitions': 2, 'biennials': 0, 'art_fairs': 0, 'museum_collections': 1, 'dealer_directory': 0, 'catalogs': 0}
df_50 저장 완료


In [18]:
all_info

[{'Artist': 'Elinor Bellingham-Smith',
  'Year': '2010',
  'Name': 'Unpopular Culture',
  'Institution': 'Mead Gallery - Warwick Arts Centre',
  'City': 'Coventry',
  'Country': 'United Kingdom',
  'Type': 'Group Exhibition',
  'url': 'https://artfacts.net/exhibition/unpopular-culture/506913'},
 {'Artist': 'Elinor Bellingham-Smith',
  'Year': '1999',
  'Name': 'Suffolk : A Female Focus',
  'Institution': 'Wolsey Art Gallery & The Room Upstairs',
  'City': 'Ipswich',
  'Country': 'United Kingdom',
  'Type': 'Group Exhibition',
  'url': 'https://artfacts.net/exhibition/suffolk-:-a-female-focus/324841'},
 {'Artist': 'Elinor Bellingham-Smith',
  'Year': '1954',
  'Name': 'Recent British Drawings',
  'Institution': 'Institute of Contemporary Arts London (ICA)',
  'City': 'London',
  'Country': 'United Kingdom',
  'Type': 'Group Exhibition',
  'url': 'https://artfacts.net/exhibition/recent-british-drawings/521672'},
 {'Artist': 'Elinor Bellingham-Smith',
  'Year': '1952',
  'Name': 'Recent T

In [20]:
# 수집한 모든 정보를 DataFrame으로 변환
# 불법 문자를 제거하는 함수
def remove_illegal_characters(text):
    if isinstance(text, str):
        return re.sub(r'[\x00-\x1F\x7F]', '', text)
    return text

# 데이터 정리하기
cleaned_info = []
for entry in all_info:
    cleaned_entry = {key: remove_illegal_characters(value) for key, value in entry.items()}
    cleaned_info.append(cleaned_entry)

# DataFrame으로 변환
df = pd.DataFrame(cleaned_info)

# DataFrame을 Excel 파일로 저장
df.to_excel('artists_exhibitions_info1.xlsx', index=False)